In [ ]:
!pip install PyVCF3
!pip install tqdm
!pip install altair

import ftplib
import pandas as pd 
import re
import vcf 
import os
import altair as alt

from glob import glob
from tqdm.notebook import tqdm

## Analyse de l'évolution des données clinvar dans le temps

Je récupère tous les VCFs de l'historique de clinvar et je fais du comptage. 
https://ftp.ncbi.nlm.nih.gov/pub/clinvar/vcf_GRCh38/archive_2.0/

In [4]:
def get_clinvar_urls(year:int)->list:
    """Récupère l'ensemble des URLS des VCF pour une date donnée"""
    SERVER = "ftp.ncbi.nlm.nih.gov"
    PATH=f"/pub/clinvar/vcf_GRCh38/archive_2.0/{year}"
    f = ftplib.FTP()
    f.connect(SERVER)
    f.login()
    f.cwd(PATH)

    dir_list = []
    f.dir(dir_list.append)

    urls = []
    for item in dir_list:
        match = re.search(r"(\w+\d+\.vcf.gz)$", item )
        if match:
            urls.append(f"{SERVER}/{PATH}/{match.group(1)}")

    return urls

In [35]:
# On génère une liste de toutes les VCFs entre 2017 et 2022

all_urls = []
for year in range(2017,2023):
    all_urls += get_clinvar_paths(year)
    
all_urls = pd.Series(all_urls)
all_urls.to_csv("clinvar_urls.csv", index=None, header=None)
all_urls

0      ftp.ncbi.nlm.nih.gov//pub/clinvar/vcf_GRCh38/a...
1      ftp.ncbi.nlm.nih.gov//pub/clinvar/vcf_GRCh38/a...
2      ftp.ncbi.nlm.nih.gov//pub/clinvar/vcf_GRCh38/a...
3      ftp.ncbi.nlm.nih.gov//pub/clinvar/vcf_GRCh38/a...
4      ftp.ncbi.nlm.nih.gov//pub/clinvar/vcf_GRCh38/a...
                             ...                        
200    ftp.ncbi.nlm.nih.gov//pub/clinvar/vcf_GRCh38/a...
201    ftp.ncbi.nlm.nih.gov//pub/clinvar/vcf_GRCh38/a...
202    ftp.ncbi.nlm.nih.gov//pub/clinvar/vcf_GRCh38/a...
203    ftp.ncbi.nlm.nih.gov//pub/clinvar/vcf_GRCh38/a...
204    ftp.ncbi.nlm.nih.gov//pub/clinvar/vcf_GRCh38/a...
Length: 205, dtype: object

In [ ]:
# On les télécharge tous en parallele ! C'est pas gros 
!cat {clinvar_urls.csv}|parallel wget 

In [37]:
!ls *vcf.gz

clinvar_20170516.vcf.gz  clinvar_20191118.vcf.gz  clinvar_20210328.vcf.gz
clinvar_20170615.vcf.gz  clinvar_20191125.vcf.gz  clinvar_20210404.vcf.gz
clinvar_20170710.vcf.gz  clinvar_20191202.vcf.gz  clinvar_20210415.vcf.gz
clinvar_20170802.vcf.gz  clinvar_20191219.vcf.gz  clinvar_20210418.vcf.gz
clinvar_20170905.vcf.gz  clinvar_20191223.vcf.gz  clinvar_20210424.vcf.gz
clinvar_20171002.vcf.gz  clinvar_20191231.vcf.gz  clinvar_20210501.vcf.gz
clinvar_20171029.vcf.gz  clinvar_20200106.vcf.gz  clinvar_20210511.vcf.gz
clinvar_20171203.vcf.gz  clinvar_20200113.vcf.gz  clinvar_20210517.vcf.gz
clinvar_20171231.vcf.gz  clinvar_20200120.vcf.gz  clinvar_20210524.vcf.gz
clinvar_20180128.vcf.gz  clinvar_20200127.vcf.gz  clinvar_20210529.vcf.gz
clinvar_20180225.vcf.gz  clinvar_20200203.vcf.gz  clinvar_20210609.vcf.gz
clinvar_20180401.vcf.gz  clinvar_20200210.vcf.gz  clinvar_20210615.vcf.gz
clinvar_20180429.vcf.gz  clinvar_20200217.vcf.gz  clinvar_20210619.vcf.gz
clinvar_20180603.vcf.gz  clinvar_20200

In [ ]:
## On parcours chaque VCF pour récupérer la date et des stats. Pour le moment que des comptage .
# A améliorer.. 
stat_df = []

for file in tqdm(glob("*vcf.gz")):
    match = re.search(r"clinvar_(\d+)\.vcf.gz", file)
    
    if match:
        date = match.group(1)
        total = !zcat {file}|grep -v "#"|wc -l 
        
        patho = !zcat {file}|grep -v "#"|grep "Pathogenic"|wc -l
        likely_path = !zcat {file}|grep -v "#"|grep "Likely_pathogenic"|wc -l
        likely_benign = !zcat {file}|grep -v "#"|grep "Likely_benign"|wc -l
        benign = !zcat {file}|grep -v "#"|grep "Benign"|wc -l
        vsi = !zcat {file}|grep -v "#"|grep "Uncertain_significance"|wc -l
        conflict = !zcat {file}|grep -v "#"|grep "Conflicting"|wc -l

        
        stat_df.append((date,int(total[0]),patho[0], likely_path[0], likely_benign[0],benign[0],vsi[0],conflict[0]))
    
            
        
stat_df = pd.DataFrame(stat_df, columns=["date","count","patho","likely_path","likely_benign","benign","vsi", "conflict"])
stat_df["date"] = pd.to_datetime(stat_df["date"])
stat_df
stat_df.to_csv("stat.csv", index=None, header=None)

In [38]:
# On anti-pivot la table et on fait un dessin 
df = stat_df.melt(id_vars="date")
alt.Chart(df).mark_line().encode(x="date",y="value",color="variable").properties(title="Nombre de variants dans clinvar")

alt.Chart(...)

In [30]:
alt.Chart(stat_df).mark_line().encode(x="date",y="count").properties(title="Nombre de variants dans clinvar")

alt.Chart(...)